In [1]:
import re
import unicodedata
import nltk

In [2]:
import os
print(os.getcwd())

C:\Users\Giantstride\Desktop\AInfinity


In [3]:
import pandas as pd

df=pd.read_csv("C:/Users/Giantstride/Desktop/AInfinity/Bad reputation.csv",encoding='latin1')

print(df.head())

                                               tweet
0  @miss_simmons1 @CooperativeBank @HSBC Donât ...
1  @HSBC_UK I wish HSBC had been able to honour t...
2  @AshDMcarp HSBC is the worst, I tried calling ...
3  @HSBC_UK If you want the worlds worst mortgage...
4  @justleeuk @cliveedwards10 @Ens_007 @HSBC_UK @...


In [4]:
data = df.astype(str) 
print(data.dtypes) 

tweet    object
dtype: object


In [5]:
import re

def clean_text(tweet):
    #Remove RT
    tweet = re.sub(r'RT', '', tweet)
    
    #Fix &
    tweet = re.sub(r'&amp;', '&', tweet)
    
    #Remove punctuations
    tweet = re.sub(r'[?!.;:,#@-]', '', tweet)

    #Convert to lowercase to maintain consistency
    tweet = tweet.lower()
    return tweet

In [6]:
data['clean_text'] = data.tweet.apply(lambda x: clean_text(x))

In [7]:
STOP_WORDS = ['a', 'about', 'above', 'after', 'again', 'against', 'all', 'also', 'am', 'an', 'and',
              'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below',
              'between', 'both', 'but', 'by', 'can', "can't", 'cannot', 'com', 'could', "couldn't", 'did',
              "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each', 'else', 'ever',
              'few', 'for', 'from', 'further', 'get', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having',
              'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how',
              "how's", 'however', 'http', 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it',
              "it's", 'its', 'itself', 'just', 'k', "let's", 'like', 'me', 'more', 'most', "mustn't", 'my', 'myself',
              'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'otherwise', 'ought', 'our', 'ours',
              'ourselves', 'out', 'over', 'own', 'r', 'same', 'shall', "shan't", 'she', "she'd", "she'll", "she's",
              'should', "shouldn't", 'since', 'so', 'some', 'such', 'than', 'that', "that's", 'the', 'their', 'theirs',
              'them', 'themselves', 'then', 'there', "there's", 'these', 'they', "they'd", "they'll", "they're",
              "they've", 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 'was', "wasn't",
              'we', "we'd", "we'll", "we're", "we've", 'were', "weren't", 'what', "what's", 'when', "when's", 'where',
              "where's", 'which', 'while', 'who', "who's", 'whom', 'why', "why's", 'with', "won't", 'would', "wouldn't",
              'www', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']

#Generate word frequency
def gen_freq(tweet):
    #Will store the list of words
    word_list = []

    #Loop over all the tweets and extract words into word_list
    for tw_words in tweet.split():
        word_list.extend(tw_words)

    #Create word frequencies using word_list
    word_freq = pd.Series(word_list).value_counts()
    
    #Drop the stopwords during the frequency calculation
    word_freq = word_freq.drop(STOP_WORDS, errors='ignore')
    
    return word_freq

#Check whether a negation term is present in the text
def any_neg(words):
    for word in words:
        if word in ['n', 'no', 'non', 'not'] or re.search(r"\wn't", word):
            return 1
    else:
        return 0

#Check whether one of the 100 rare words is present in the text
def any_rare(words, rare_100):
    for word in words:
        if word in rare_100:
            return 1
    else:
        return 0

#Check whether prompt words are present
def is_question(words):
    for word in words:
        if word in ['when', 'what', 'how', 'why', 'who']:
            return 1
    else:
        return 0

In [8]:
word_freq = gen_freq(data.clean_text.str)
#100 most rare words in the dataset
rare_100 = word_freq[-100:]
#Number of words in a tweet
data['word_count'] = data.clean_text.str.split().apply(lambda x: len(x))
#Negation present or not
data['any_neg'] = data.clean_text.str.split().apply(lambda x: any_neg(x))
#Prompt present or not
data['is_question'] = data.clean_text.str.split().apply(lambda x: is_question(x))
#Any of the most 100 rare words present or not
data['any_rare'] = data.clean_text.str.split().apply(lambda x: any_rare(x, rare_100))
#Character count of the tweet
data['char_count'] = data.clean_text.apply(lambda x: len(x))

In [9]:
#Top 10 common words are
gen_freq(data.clean_text.str)[:50]

hsbc              70
hsbc_uk           45
worst             39
service           26
customer          24
bank              21
awful             19
worse             15
banks             12
applied           11
&                 11
time              11
account           11
help              11
money             11
even              11
will              10
disappointed      10
years             10
now                9
banking            9
hsbcukbusiness     9
waiting            9
business           9
got                8
switch             8
us                 8
weeks              8
know               8
today              7
barclays           7
card               7
customers          7
ago                7
bbl                7
hold               7
seems              6
first              6
never              6
trying             6
still              6
starlingbank       6
hsbc_in            6
canât            6
day                6
app                6
make               5
lot          

In [10]:
words = gen_freq(''.join(str(df['tweet'].tolist())))

In [11]:
(pd.Series(nltk.ngrams(words, 2)).value_counts())[:10]

(1, 1)        11
(12, 12)       6
(3, 3)         3
(16, 16)       2
(197, 192)     1
(24, 24)       1
(19, 18)       1
(166, 156)     1
(7, 6)         1
(35, 33)       1
dtype: int64